In [43]:
!pip install pytorch-lightning

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 825.4/825.4 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 43.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 40.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 43.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 963.5/963.5 kB 52.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-c

In [44]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import random
import cmath
import torch.nn.functional as F
from torch.optim.swa_utils import AveragedModel, SWALR
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor



In [2]:
# Количество итераций
NUM_SAMPLES = 10000

# Длина нити
l = 1.0

# Генерация случайных параметров в допустимых пределах
def generate_params():
    while True:
        x1 = random.uniform(0.1, 0.45)
        x2 = random.uniform(0.55, 0.9)
        if x1 < x2 < l:
            break
    return {
        'T': random.uniform(0.5, 5.0),
        'x1': x1,
        'x2': x2,
        'm1': random.uniform(0.1, 5.0),
        'm2': random.uniform(0.1, 5.0)
    }

In [3]:
results = []

for _ in range(NUM_SAMPLES):
    params = generate_params()
    T = params['T']
    x1 = params['x1']
    x2 = params['x2']
    m1 = params['m1']
    m2 = params['m2']

    try:
        # Расчёт коэффициентов
        W1 = T / (m1 * (x2 - x1))
        W2 = T / (m2 * (x2 - x1))
        Q1 = T / (m1 * x1)
        Q2 = T / (m2 * (l - x2))

        # Характеристическое уравнение: λ⁴ + aλ² + b = 0
        a = W1 + W2 + Q1 + Q2
        b = W1 * Q2 + W2 * Q1 + Q1 * Q2

        D = a**2 - 4 * b

        # Комплексные корни через cmath
        sqrt_D = cmath.sqrt(D)
        lambda_sq1 = (-a + sqrt_D) / 2
        lambda_sq2 = (-a - sqrt_D) / 2

        omega1 = cmath.sqrt(lambda_sq1)
        omega2 = cmath.sqrt(lambda_sq2)

        # Углы отклонения в узлах (при y = 1, нормированная амплитуда)
        theta1_omega1 = (m1 * (omega1**2).real) / T
        theta2_omega1 = (m2 * (omega1**2).real) / T

        theta1_omega2 = (m1 * (omega2**2).real) / T
        theta2_omega2 = (m2 * (omega2**2).real) / T

        # Нормализация углов по модулю 2π
        mod2pi = lambda angle: angle % (2 * np.pi)

        theta1_omega1 = mod2pi(theta1_omega1)
        theta2_omega1 = mod2pi(theta2_omega1)
        theta1_omega2 = mod2pi(theta1_omega2)
        theta2_omega2 = mod2pi(theta2_omega2)
        results.append({
            'T': T,
            'x1': x1,
            'x2': x2,
            'm1': m1,
            'm2': m2,
            'omega1': str(omega1),
            'omega2': str(omega2),
            'theta1_omega1': theta1_omega1,
            'theta2_omega1': theta2_omega1,
            'theta1_omega2': theta1_omega2,
            'theta2_omega2': theta2_omega2
        })

    except Exception:
        continue  # безопасный пропуск ошибки

# Создание DataFrame
df = pd.DataFrame(results)
# Порог для сравнения с плавающей точкой
epsilon = 1e-6

# Условия фильтрации
cond_T = np.abs(df["T"] - 1.0) < epsilon
cond_X = np.abs(df["x1"] + df["x2"] - 1.0) < epsilon
cond_M = np.abs(df["m1"] - df["m2"]) < epsilon

# Все комбинации фильтров
filters = {
    "FILTER_T_equals_1": cond_T,
    "FILTER_X_symmetric": cond_X,
    "FILTER_M_equal": cond_M,
    "FILTER_T_and_X": cond_T & cond_X,
    "FILTER_T_and_M": cond_T & cond_M,
    "FILTER_X_and_M": cond_X & cond_M,
    "FILTER_T_and_X_and_M": cond_T & cond_X & cond_M,
}

# Сохраняем каждую таблицу
for name, condition in filters.items():
    filtered = df[condition]
    if not filtered.empty:
        filtered.to_csv(f"{name}.csv", index=False)
        filtered.to_parquet(f"{name}.parquet", index=False)
        print(f"✅ Сохранено {len(filtered)} строк в {name}.csv и {name}.parquet")
    else:
        print(f"⚠️ Нет данных для {name}")

# Сохранение в CSV и Parquet
df.to_csv("TEST_TABLE.csv", index=False)
df.to_parquet("TEST_TABLE.parquet", index=False)

print(f"✅ Успешно сохранено {len(df)} решений в файлы с углами отклонения:")
print(" - TEST_TABLE.csv")
print(" - TEST_TABLE.parquet")

⚠️ Нет данных для FILTER_T_equals_1
⚠️ Нет данных для FILTER_X_symmetric
⚠️ Нет данных для FILTER_M_equal
⚠️ Нет данных для FILTER_T_and_X
⚠️ Нет данных для FILTER_T_and_M
⚠️ Нет данных для FILTER_X_and_M
⚠️ Нет данных для FILTER_T_and_X_and_M
✅ Успешно сохранено 10000 решений в файлы с углами отклонения:
 - TEST_TABLE.csv
 - TEST_TABLE.parquet


In [4]:
# Сохранение в CSV и Parquet
df.to_csv("TEST_TABLE.csv", index=False)
df.to_parquet("TEST_TABLE.parquet", index=False)

print(f"✅ Успешно сохранено {len(df)} решений в файлы с углами отклонения:")
print(" - TEST_TABLE.csv")
print(" - TEST_TABLE.parquet")

✅ Успешно сохранено 10000 решений в файлы с углами отклонения:
 - TEST_TABLE.csv
 - TEST_TABLE.parquet


In [5]:
df.head(10)

,T,x1,x2,m1,m2,omega1,omega2,theta1_omega1,theta2_omega1,theta1_omega2,theta2_omega2
0,1.104709,0.399127,0.812554,4.346115,4.082862,0.9551391156945878j,1.5611867625667124j,2.694076,2.911476,2.977595,3.558407
1,4.268538,0.217337,0.744947,1.768638,3.649265,2.403900621612623j,4.0869738223261285j,3.888807,1.342818,5.645457,4.569499
2,3.503599,0.128819,0.749599,3.311316,0.418718,3.0507063409622908j,6.892481983559633j,3.770333,5.170920,5.366389,0.605663
3,4.232558,0.385680,0.793757,2.862446,1.114176,2.431305780399946j,5.410578536350306j,2.285454,4.727111,5.334719,4.860203
4,2.816420,0.228333,0.838827,4.206653,3.918657,1.854957045690182j,2.4949133450374954j,1.143850,1.495698,3.269213,3.905714
5,2.758565,0.350953,0.747448,1.044914,3.816722,1.8824583182985526j,3.914295891578514j,4.940890,1.380228,0.479491,3.933776
6,0.729009,0.183804,0.563470,1.230856,3.849184,0.8603388988185193j,2.23043873487507j,5.033463,2.375005,4.166843,5.148586
7,0.752987,0.248461,0.725130,2.452878,3.156656,1.0006477185396856j,1.498834911968927j,3.021436,2.085577,5.248314,3.148621
8,2.407876,0.442737,0.674469,1.663849,2.862138,1.6808078588449211j,3.592128075129358j,4.331023,2.925094,3.650097,3.511864
9,1.670080,0.130997,0.749820,4.692820,4.901927,1.3084186143296535j,1.8687451346497028j,1.472687,1.258336,2.753483,2.316231


In [6]:
df['omega1'] = df['omega1'].apply(lambda s: abs(complex(s)))
df['omega2'] = df['omega2'].apply(lambda s: abs(complex(s)))

In [7]:
df[["T", "omega1", "omega2", "theta1_omega1", "theta2_omega1", "theta1_omega2", "theta2_omega2"]].head()

,T,omega1,omega2,theta1_omega1,theta2_omega1,theta1_omega2,theta2_omega2
0,1.104709,0.955139,1.561187,2.694076,2.911476,2.977595,3.558407
1,4.268538,2.403901,4.086974,3.888807,1.342818,5.645457,4.569499
2,3.503599,3.050706,6.892482,3.770333,5.170920,5.366389,0.605663
3,4.232558,2.431306,5.410579,2.285454,4.727111,5.334719,4.860203
4,2.816420,1.854957,2.494913,1.143850,1.495698,3.269213,3.905714


In [8]:
df[['x1','x2',	'm1',	'm2']].head()

,x1,x2,m1,m2
0,0.399127,0.812554,4.346115,4.082862
1,0.217337,0.744947,1.768638,3.649265
2,0.128819,0.749599,3.311316,0.418718
3,0.385680,0.793757,2.862446,1.114176
4,0.228333,0.838827,4.206653,3.918657


In [9]:
#x1	x2	m1	m2 - наш таргет
X_train, X_test, y_train, y_test = train_test_split(df[["T", "omega1", "omega2", "theta1_omega1", "theta2_omega1", "theta1_omega2", "theta2_omega2"]],
                                                    df[['x1','x2',	'm1',	'm2']], test_size=0.2, random_state=42)

In [10]:
X_train.head()

,T,omega1,omega2,theta1_omega1,theta2_omega1,theta1_omega2,theta2_omega2
9254,3.087715,3.691574,4.668083,5.520796,2.007698,1.300354,5.729780
1561,0.937806,0.890160,1.447794,2.528559,3.478093,2.634188,5.146007
1670,4.728534,2.451400,3.750739,2.831981,2.742491,4.487027,4.277527
6087,4.225281,2.143960,3.574873,3.655602,2.201307,5.260965,1.217627
6669,1.677160,1.652114,2.710769,5.735706,4.325548,0.460131,1.012861


In [13]:
X_test.iloc[0]

,6252
T,4.747499
omega1,2.160740
omega2,4.035654
theta1_omega1,3.154813
theta2_omega1,2.058045
theta1_omega2,1.653443
theta2_omega2,4.110693


In [14]:
scaler_X = StandardScaler().fit(X_train)
scaler_y = StandardScaler().fit(y_train)

X_train = scaler_X.transform(X_train)
X_test = scaler_X.transform(X_test)
y_train = scaler_y.transform(y_train)
y_test = scaler_y.transform(y_test)

# Преобразование в тензоры PyTorch
X_train_t = torch.tensor(X_train, dtype=torch.float32)
y_train_t = torch.tensor(y_train, dtype=torch.float32)
X_test_t = torch.tensor(X_test, dtype=torch.float32)
y_test_t = torch.tensor(y_test, dtype=torch.float32)

# Создание DataLoader
train_dataset = TensorDataset(X_train_t, y_train_t)
test_dataset = TensorDataset(X_test_t, y_test_t)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64)

In [15]:
class PhysicsNet(nn.Module):
    def __init__(self, input_size, output_size):
        super(PhysicsNet, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(input_size, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.3),

            nn.Linear(256, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(0.4),

            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.3),

            nn.Linear(256, 128),
            nn.ReLU(),

            nn.Linear(128, output_size)
        )

    def forward(self, x):
        return self.net(x)

# Инициализация модели
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = PhysicsNet(input_size=7, output_size=4).to(device)

# Функция потерь и оптимизатор
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode='min', factor=0.5, patience=5, verbose=True
)


/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


In [18]:
def train_model(model, train_loader, test_loader, epochs=100):
    train_losses = []
    test_losses = []

    for epoch in range(epochs):
        # Режим обучения
        model.train()
        running_loss = 0.0

        for inputs, targets in train_loader:
            inputs, targets = inputs.to(device), targets.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)

        epoch_loss = running_loss / len(train_loader.dataset)
        train_losses.append(epoch_loss)

        # Валидация
        model.eval()
        test_loss = 0.0
        with torch.no_grad():
            for inputs, targets in test_loader:
                inputs, targets = inputs.to(device), targets.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, targets)
                test_loss += loss.item() * inputs.size(0)

        test_loss /= len(test_loader.dataset)
        test_losses.append(test_loss)
        scheduler.step(test_loss)

        print(f'Epoch {epoch+1}/{epochs} | '
              f'Train Loss: {epoch_loss:.6f} | '
              f'Test Loss: {test_loss:.6f}')

    # График обучения
    plt.figure(figsize=(10, 6))
    plt.plot(train_losses, label='Train Loss')
    plt.plot(test_losses, label='Test Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.title('Training and Validation Loss')
    plt.savefig('training_loss.png')
    plt.close()
    #blob:https://colab.research.google.com/193f8bbb-07d3-4956-ae63-b2999c8ec532
    return model

In [19]:
trained_model = train_model(model, train_loader, test_loader, epochs=200)

# Сохранение модели
torch.save(trained_model.state_dict(), 'physics_net_model.pth')

# Функция для предсказания
def predict(input_data):
    """Предсказывает параметры системы по входным данным"""
    # Преобразование входных данных
    input_scaled = scaler_X.transform(np.array([input_data]))
    input_tensor = torch.tensor(input_scaled, dtype=torch.float32).to(device)

    # Предсказание
    trained_model.eval()
    with torch.no_grad():
        output = trained_model(input_tensor)

    # Обратное масштабирование
    predicted = scaler_y.inverse_transform(output.cpu().numpy())
    return {
        'x1': predicted[0][0],
        'x2': predicted[0][1],
        'm1': predicted[0][2],
        'm2': predicted[0][3]
    }

# Пример использования
example_input = [1.605, 1.335, 3.984, 1.97, 5.58, 5.55, 0.024]
prediction = predict(example_input)
print("Predicted parameters:")
print(f"x1: {prediction['x1']:.4f}, x2: {prediction['x2']:.4f}")
print(f"m1: {prediction['m1']:.4f}, m2: {prediction['m2']:.4f}")

Epoch 1/200 | Train Loss: 0.567529 | Test Loss: 0.383588
Epoch 2/200 | Train Loss: 0.448930 | Test Loss: 0.332457
Epoch 3/200 | Train Loss: 0.409668 | Test Loss: 0.301391
Epoch 4/200 | Train Loss: 0.381036 | Test Loss: 0.284728
Epoch 5/200 | Train Loss: 0.368365 | Test Loss: 0.271348
Epoch 6/200 | Train Loss: 0.357915 | Test Loss: 0.274113
Epoch 7/200 | Train Loss: 0.346560 | Test Loss: 0.259342
Epoch 8/200 | Train Loss: 0.340381 | Test Loss: 0.250786
Epoch 9/200 | Train Loss: 0.328309 | Test Loss: 0.247212
Epoch 10/200 | Train Loss: 0.323833 | Test Loss: 0.249311
Epoch 11/200 | Train Loss: 0.314724 | Test Loss: 0.232008
Epoch 12/200 | Train Loss: 0.309226 | Test Loss: 0.229034
Epoch 13/200 | Train Loss: 0.310218 | Test Loss: 0.226976
Epoch 14/200 | Train Loss: 0.304951 | Test Loss: 0.219240
Epoch 15/200 | Train Loss: 0.299969 | Test Loss: 0.225737
Epoch 16/200 | Train Loss: 0.294421 | Test Loss: 0.227976
Epoch 17/200 | Train Loss: 0.293964 | Test Loss: 0.214977
Epoch 18/200 | Train Lo

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [20]:
torch.save(trained_model.state_dict(), 'physics_net_model.pth')

In [21]:
def predict(input_data):
    """Предсказывает параметры системы по входным данным"""
    # Преобразование входных данных
    input_scaled = scaler_X.transform(np.array([input_data]))
    input_tensor = torch.tensor(input_scaled, dtype=torch.float32).to(device)

    # Предсказание
    trained_model.eval()
    with torch.no_grad():
        output = trained_model(input_tensor)

    # Обратное масштабирование
    predicted = scaler_y.inverse_transform(output.cpu().numpy())
    return {
        'x1': predicted[0][0],
        'x2': predicted[0][1],
        'm1': predicted[0][2],
        'm2': predicted[0][3]
    }

# Пример использования
example_input = [1.129299,	1.225282,	2.093613,	3.086891,	2.340132,	3.234530,	1.054305]
prediction = predict(example_input)
print("Predicted parameters:")
print(f"x1: {prediction['x1']:.4f}, x2: {prediction['x2']:.4f}")
print(f"m1: {prediction['m1']:.4f}, m2: {prediction['m2']:.4f}")

Predicted parameters:
x1: 0.1549, x2: 0.6047
m1: 2.4718, m2: 2.9212


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [23]:
class EnhancedPhysicsNet(nn.Module):
    def __init__(self, input_size, output_size):
        super(EnhancedPhysicsNet, self).__init__()
        self.fc1 = nn.Linear(input_size, 512)
        self.bn1 = nn.BatchNorm1d(512)

        self.res_block1 = nn.Sequential(
            nn.Linear(512, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(512, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(0.4)
        )

        self.attention = nn.Sequential(
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 512),
            nn.Sigmoid()
        )

        self.fc2 = nn.Linear(512, 256)
        self.bn2 = nn.BatchNorm1d(256)

        self.fc3 = nn.Linear(256, 128)
        self.output = nn.Linear(128, output_size)

    def forward(self, x):
        x = F.relu(self.bn1(self.fc1(x)))

        # Residual block
        residual = x
        x = self.res_block1(x)
        x = x + residual

        # Attention mechanism
        attn_weights = self.attention(x)
        x = x * attn_weights

        x = F.relu(self.bn2(self.fc2(x)))
        x = F.relu(self.fc3(x))
        return self.output(x)

In [32]:
def train_enhanced_model(model, train_loader, val_loader, epochs=300):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)

    # Комбинированная функция потерь
    criterion = nn.MSELoss()
    optimizer = optim.AdamW(model.parameters(), lr=0.001, weight_decay=1e-4)

    scheduler = optim.lr_scheduler.OneCycleLR(
        optimizer,
        max_lr=0.01,
        epochs=epochs,
        steps_per_epoch=len(train_loader),
        anneal_strategy='cos'
    )

    early_stop = EarlyStopping(patience=20, verbose=True)

    for epoch in range(epochs):
        model.train()
        train_loss = 0.0

        for inputs, targets in train_loader:
            inputs, targets = inputs.to(device), targets.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)

            # Основная функция потерь
            loss = criterion(outputs, targets)

            # Физические ограничения
            x1 = outputs[:, 0]
            x2 = outputs[:, 1]
            m1 = outputs[:, 2]
            m2 = outputs[:, 3]

            # Штраф за нарушение порядка x1 < x2
            order_penalty = torch.mean(torch.relu(x1 - x2))

            # Штраф за отрицательные массы
            mass_penalty = torch.mean(torch.relu(-m1) + torch.mean(torch.relu(-m2)))

            # Комбинированная потеря
            total_loss = loss + 0.1 * order_penalty + 0.1 * mass_penalty
            total_loss.backward()

            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            scheduler.step()

            train_loss += total_loss.item()

        # Валидация
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for inputs, targets in val_loader:
                inputs, targets = inputs.to(device), targets.to(device)
                outputs = model(inputs)
                val_loss += criterion(outputs, targets).item()

        train_loss /= len(train_loader)
        val_loss /= len(val_loader)

        print(f'Epoch {epoch+1}/{epochs} | Train Loss: {train_loss:.6f} | Val Loss: {val_loss:.6f}')

        # Ранняя остановка
        early_stop(val_loss, model)
        if early_stop.early_stop:
            print("Early stopping")
            break

    model.load_state_dict(torch.load('checkpoint.pt'))
    return model

In [33]:
def prepare_enhanced_data(file_path):
    data = pd.read_csv(file_path)

    # Преобразование комплексных частот
    data['omega1'] = data['omega1'].apply(lambda s: abs(complex(s)))
    data['omega2'] = data['omega2'].apply(lambda s: abs(complex(s)))

    # Создание новых признаков
    data['omega_ratio'] = data['omega1'] / data['omega2']
    data['theta_ratio1'] = data['theta1_omega1'] / data['theta2_omega1']
    data['theta_ratio2'] = data['theta1_omega2'] / data['theta2_omega2']
    data['T_omega1'] = data['T'] * data['omega1']
    data['T_omega2'] = data['T'] * data['omega2']

    # Выбор признаков
    features = data[[
        "T", "omega1", "omega2",
        "theta1_omega1", "theta2_omega1",
        "theta1_omega2", "theta2_omega2",
        "omega_ratio", "theta_ratio1", "theta_ratio2",
        "T_omega1", "T_omega2"
    ]]

    targets = data[["x1", "x2", "m1", "m2"]]

    return features.values, targets.values

In [34]:
def generate_more_data(base_file, scale=5):
    base_data = pd.read_csv(base_file)
    expanded_data = pd.DataFrame()

    # Convert complex numbers to absolute values before adding noise
    base_data['omega1'] = base_data['omega1'].apply(lambda s: abs(complex(s)))
    base_data['omega2'] = base_data['omega2'].apply(lambda s: abs(complex(s)))

    for i in range(scale):
        # Добавление шума
        noisy_data = base_data.copy()
        for col in ['T', 'omega1', 'omega2', 'theta1_omega1', 'theta2_omega1',
                   'theta1_omega2', 'theta2_omega2']:
            if col in ['omega1', 'omega2']:
                # Меньший шум для частот
                noise_level = 0.01
            else:
                noise_level = 0.05

            noisy_data[col] = noisy_data[col] * (1 + noise_level * np.random.randn(len(noisy_data)))

        expanded_data = pd.concat([expanded_data, noisy_data])

    # Добавление физически валидных вариаций
    for _ in range(scale//2):
        variant = base_data.copy()
        variant['x1'] = np.clip(variant['x1'] * (1 + 0.1 * np.random.randn(len(variant))), 0.1, 0.45)
        variant['x2'] = np.clip(variant['x2'] * (1 + 0.1 * np.random.randn(len(variant))), 0.55, 0.9)
        variant['m1'] = np.abs(variant['m1'] * (1 + 0.2 * np.random.randn(len(variant))))
        variant['m2'] = np.abs(variant['m2'] * (1 + 0.2 * np.random.randn(len(variant))))
        expanded_data = pd.concat([expanded_data, variant])

    return expanded_data

In [36]:
# Define the EarlyStopping class
class EarlyStopping:
    def __init__(self, patience=7, verbose=False, delta=0, path='checkpoint.pt', trace_func=print):
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.inf
        self.delta = delta
        self.path = path
        self.trace_func = trace_func

    def __call__(self, val_loss, model):
        score = -val_loss
        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            self.trace_func(f'EarlyStopping counter: {self.counter} of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        if self.verbose:
            self.trace_func(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        torch.save(model.state_dict(), self.path)
        self.val_loss_min = val_loss

# Define the test_model function
def test_model(model, test_loader, scaler_y):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.eval()
    predictions = []
    actuals = []
    with torch.no_grad():
        for inputs, targets in test_loader:
            inputs = inputs.to(device)
            outputs = model(inputs)
            predictions.append(outputs.cpu().numpy())
            actuals.append(targets.cpu().numpy())

    predictions = np.concatenate(predictions)
    actuals = np.concatenate(actuals)

    # Inverse transform
    predictions_inv = scaler_y.inverse_transform(predictions)
    actuals_inv = scaler_y.inverse_transform(actuals)

    # Evaluate
    mse = np.mean((predictions_inv - actuals_inv)**2)
    print(f"Test MSE: {mse:.6f}")

    # You can add more evaluation metrics here if needed
    # For example, R2 score:
    # from sklearn.metrics import r2_score
    # r2 = r2_score(actuals_inv, predictions_inv)
    # print(f"Test R2: {r2:.6f}")


# 1. Генерация дополнительных данных
expanded_data = generate_more_data("TEST_TABLE.csv", scale=5)
expanded_data.to_csv("ENHANCED_DATA.csv", index=False)

# 2. Подготовка данных с новыми признаками
X, y = prepare_enhanced_data("ENHANCED_DATA.csv")

# 3. Разделение данных
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# 4. Масштабирование
scaler_X = StandardScaler().fit(X_train)
scaler_y = StandardScaler().fit(y_train)

X_train = scaler_X.transform(X_train)
X_val = scaler_X.transform(X_val)
X_test = scaler_X.transform(X_test)
y_train = scaler_y.transform(y_train)
y_val = scaler_y.transform(y_val)
y_test = scaler_y.transform(y_test)

# 5. Создание DataLoader
train_dataset = TensorDataset(torch.tensor(X_train, dtype=torch.float32),
                             torch.tensor(y_train, dtype=torch.float32))
val_dataset = TensorDataset(torch.tensor(X_val, dtype=torch.float32),
                           torch.tensor(y_val, dtype=torch.float32))
test_dataset = TensorDataset(torch.tensor(X_test, dtype=torch.float32),
                            torch.tensor(y_test, dtype=torch.float32))

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=128)
test_loader = DataLoader(test_dataset, batch_size=128)

# 6. Инициализация и обучение модели
model = EnhancedPhysicsNet(input_size=X_train.shape[1], output_size=4)
trained_model = train_enhanced_model(model, train_loader, val_loader, epochs=300)

# 7. Оценка модели
test_model(trained_model, test_loader, scaler_y)

Epoch 1/300 | Train Loss: 0.499074 | Val Loss: 0.319590
Validation loss decreased (inf --> 0.319590).  Saving model ...
Epoch 2/300 | Train Loss: 0.414869 | Val Loss: 0.274768
Validation loss decreased (0.319590 --> 0.274768).  Saving model ...
Epoch 3/300 | Train Loss: 0.390632 | Val Loss: 0.260480
Validation loss decreased (0.274768 --> 0.260480).  Saving model ...
Epoch 4/300 | Train Loss: 0.377426 | Val Loss: 0.260526
EarlyStopping counter: 1 of 20
Epoch 5/300 | Train Loss: 0.368391 | Val Loss: 0.240967
Validation loss decreased (0.260480 --> 0.240967).  Saving model ...
Epoch 6/300 | Train Loss: 0.360205 | Val Loss: 0.248384
EarlyStopping counter: 1 of 20
Epoch 7/300 | Train Loss: 0.351025 | Val Loss: 0.234975
Validation loss decreased (0.240967 --> 0.234975).  Saving model ...
Epoch 8/300 | Train Loss: 0.346534 | Val Loss: 0.237842
EarlyStopping counter: 1 of 20
Epoch 9/300 | Train Loss: 0.343717 | Val Loss: 0.228662
Validation loss decreased (0.234975 --> 0.228662).  Saving mode

In [37]:
torch.save({
    'model_state_dict': trained_model.state_dict(),
    'scaler_X': scaler_X,
    'scaler_y': scaler_y
}, "enhanced_physics_model.pth")

In [40]:
class EnhancedPhysicsPredictor:
    def __init__(self, model_path="enhanced_physics_model.pth"):
        # Загрузка модели и масштабировщиков
        checkpoint = torch.load(model_path, map_location=torch.device('cpu'), weights_only=False)

        # Определение архитектуры модели
        self.model = EnhancedPhysicsNet(input_size=12, output_size=4)
        self.model.load_state_dict(checkpoint['model_state_dict'])
        self.model.eval()

        # Загрузка масштабировщиков
        self.scaler_X = checkpoint['scaler_X']
        self.scaler_y = checkpoint['scaler_y']

    def prepare_input(self, input_data):
        """Подготавливает входные данные, добавляя новые признаки"""
        # Базовые признаки
        T = input_data[0]
        omega1 = input_data[1]
        omega2 = input_data[2]
        theta1_omega1 = input_data[3]
        theta2_omega1 = input_data[4]
        theta1_omega2 = input_data[5]
        theta2_omega2 = input_data[6]

        # Новые признаки
        omega_ratio = omega1 / omega2
        theta_ratio1 = theta1_omega1 / theta2_omega1
        theta_ratio2 = theta1_omega2 / theta2_omega2
        T_omega1 = T * omega1
        T_omega2 = T * omega2

        # Собираем все 12 признаков
        return np.array([
            T, omega1, omega2,
            theta1_omega1, theta2_omega1,
            theta1_omega2, theta2_omega2,
            omega_ratio, theta_ratio1, theta_ratio2,
            T_omega1, T_omega2
        ])

    def apply_physical_constraints(self, prediction):
        """Применяет физические ограничения к предсказаниям"""
        x1, x2, m1, m2 = prediction

        # Гарантируем x1 < x2
        if x1 >= x2:
            # Корректируем с сохранением среднего
            avg = (x1 + x2) / 2
            diff = abs(x1 - x2) * 0.1  # 10% от разницы
            x1 = avg - diff
            x2 = avg + diff

        # Гарантируем положительные массы
        m1 = max(0.01, m1)
        m2 = max(0.01, m2)

        # Гарантируем физические пределы
        x1 = np.clip(x1, 0.05, 0.49)
        x2 = np.clip(x2, 0.51, 0.95)

        return [x1, x2, m1, m2]

    def predict(self, input_data):
        """
        Предсказывает параметры системы по входным данным
        Вход: список из 7 значений [T, omega1, omega2, theta1_omega1, theta2_omega1, theta1_omega2, theta2_omega2]
        """
        # Подготовка входных данных
        prepared_input = self.prepare_input(input_data)

        # Масштабирование
        scaled_input = self.scaler_X.transform(prepared_input.reshape(1, -1))
        input_tensor = torch.tensor(scaled_input, dtype=torch.float32)

        # Предсказание
        with torch.no_grad():
            scaled_output = self.model(input_tensor).numpy()

        # Обратное масштабирование
        prediction = self.scaler_y.inverse_transform(scaled_output)[0]

        # Применение физических ограничений
        constrained_pred = self.apply_physical_constraints(prediction)

        return {
            'x1': constrained_pred[0],
            'x2': constrained_pred[1],
            'm1': constrained_pred[2],
            'm2': constrained_pred[3],
            'raw_prediction': prediction.tolist()  # Для отладки
        }

In [41]:
predictor = EnhancedPhysicsPredictor()


input_example = [1.605, 1.335, 3.984, 1.97, 5.58, 5.55, 0.024]

result = predictor.predict(input_example)

print("\n" + "="*50)
print("Улучшенное предсказание параметров системы:")
print(f"Позиция первой массы (x1): {result['x1']:.4f}")
print(f"Позиция второй массы (x2): {result['x2']:.4f}")
print(f"Величина первой массы (m1): {result['m1']:.4f}")
print(f"Величина второй массы (m2): {result['m2']:.4f}")
print("="*50)


Улучшенное предсказание параметров системы:
Позиция первой массы (x1): 0.2902
Позиция второй массы (x2): 0.6693
Величина первой массы (m1): 3.9092
Величина второй массы (m2): 0.9070
